In [4]:
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [2]:
import copy
from konlpy.tag import Okt
import pykospacing
import kss
from nltk.tokenize import TreebankWordTokenizer
from wordcloud import WordCloud
#불용어를 제거하기 위해 만든 stopwords 파일을 읽어옵니다.
with open('data/stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
#stopword 파일을 작성할 때 썼던 줄바꿈 문자들을 전부 제거합니다.
stopwords = [x.replace('\n','') for x in stopwords]
okt = Okt()
tokenizer = TreebankWordTokenizer()
def word_preprocess_korean(text):
    my_text = copy.copy(text)
    #먼저 줄바꿈 문자들을 전부 제거합니다.
    my_text = my_text.replace('\n', '')
    spacer = pykospacing.Spacing()
    #이 후 띄어쓰기가 잘못된 경우 전부 다 수정합니다.
    my_text = spacer(my_text) 
    #본문 내용에 대해 각 문장으로 분리합니다.
    sents = kss.split_sentences(my_text)
    result = []
    for sent in sents:
        #각 문장에 대해서 품사를 기준으로 단어들을 분리합니다.
        morphs = okt.pos(sent)
        for word, pos in morphs:
            #명사이면서 단어의 길이가 1보다 큰 단어들만 리스트에 포함시킵니다.
            if pos in ['Noun'] and len(word) >= 2:
                result.append(word)
    return result

In [6]:
df_dust = pd.read_csv('지구온난화_.csv',encoding = 'utf-8')

In [7]:
# lda_preprocessed_document == word_preprocess_korean (사용자 정의 함수)
df_dust['lda_preprocessed_document'] = df_dust['content'].apply(lambda x: word_preprocess_korean(x))
dust = ''
# dataframe의 content(본문) 부분이 list 형태이기 때문에 이를 str로 변환
for lst in df_dust['lda_preprocessed_document']:
    dust = " ".join(lst)   
from gensim import corpora
import gensim
from IPython.core.display import HTML
import pyLDAvis
from pyLDAvis import gensim_models
# 전처리가 된 context에 대해서 진행
word_dict = corpora.Dictionary(df_dust['lda_preprocessed_document'])
# Bag of words를 ㅌ`사용해서 벡터화를 시킴
corpus = [word_dict.doc2bow(text) for text in df_dust['lda_preprocessed_document']]
# topic을 15개로 지정
N_TOPICS = 15
# gensim 라이브러를 활용해서 LDA model 만들고 ldamodel에 저장
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics = N_TOPICS, 
                                           id2word=word_dict)
# 시각화
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, word_dict)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D

